# CNN con PyTorch - Clasificación Cats vs Dogs

Este notebook entrena un modelo sencillo de red neuronal convolucional para clasificar imágenes de gatos y perros.


## Importaciones y preparacion de Datos

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# Transformaciones: normalización y tensor
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Cargar dataset MNIST
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


## Definicion de la CNN

In [20]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)   # 32 filtros, kernel 3x3
        self.conv2 = nn.Conv2d(32, 64, 3, 1) # 64 filtros
        self.dropout1 = nn.Dropout(0.25)
        self.fc1 = nn.Linear(9216, 128)      # Capa fully connected
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x


## Entrenamiento y Evaluacion de la CNN

In [21]:
model_cnn = SimpleCNN().to(device)
optimizer = optim.Adam(model_cnn.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

def train(model, loader, optimizer, criterion, epochs=5):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(loader):.4f}")

def test(model, loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    acc = 100. * correct / len(loader.dataset)
    print(f"Accuracy: {acc:.2f}%")
    return acc

# Entrenamos la CNN
train(model_cnn, train_loader, optimizer, criterion, epochs=5)
acc_cnn = test(model_cnn, test_loader)


KeyboardInterrupt: 

## Definicion del MLP

In [ ]:
class SimpleMLP(nn.Module):
    def __init__(self):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(28*28, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


## Entrenamiento y Evaluacion del MLP

In [ ]:
model_mlp = SimpleMLP().to(device)
optimizer_mlp = optim.Adam(model_mlp.parameters(), lr=0.001)

train(model_mlp, train_loader, optimizer_mlp, criterion, epochs=5)
acc_mlp = test(model_mlp, test_loader)


Epoch 1, Loss: 0.3462
Epoch 2, Loss: 0.1572
Epoch 3, Loss: 0.1138
Epoch 4, Loss: 0.0928
Epoch 5, Loss: 0.0750
Accuracy: 96.94%


## Comparacion Final

In [ ]:
print(f"Accuracy CNN: {acc_cnn:.2f}%")
print(f"Accuracy MLP: {acc_mlp:.2f}%")

if acc_cnn > acc_mlp:
    print("La CNN tuvo mejor desempeño.")
else:
    print("El MLP tuvo mejor desempeño.")


Accuracy CNN: 98.89%
Accuracy MLP: 96.94%
La CNN tuvo mejor desempeño.
